In [0]:
from __future__ import absolute_import, division, print_function, unicode_literals

# Install TensorFlow
try:
  # %tensorflow_version only works in Colab.
  %tensorflow_version 2.x
except Exception:
  pass

import tensorflow as tf

In [0]:
#import sys
#sys.path.append('/content/drive/My Drive/SP1_2020/autoencoder/')

In [0]:
# import the necessary packages
from tensorflow.keras.layers import BatchNormalization
from tensorflow.keras.layers import Conv2D
from tensorflow.keras.layers import Conv2DTranspose
from tensorflow.keras.layers import LeakyReLU
from tensorflow.keras.layers import Activation
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Reshape
from tensorflow.keras.layers import Input
from tensorflow.keras.models import Model
from tensorflow.keras import backend as K
import numpy as np
from google.colab.patches import cv2_imshow

In [30]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [0]:
class ConvAutoencoder:
    @staticmethod
    def build(width, height, depth, filters=(32, 64), latentDim=16):
        # initialize the input shape to be "channels last" along with
        # the channels dimension itself
        # channels dimension itself
        inputShape = (height, width, depth)
        chanDim = -1

        # define the input to the encoder
        inputs = Input(shape=inputShape)
        x = inputs

        # loop over the number of filters
        for f in filters:
            # apply a CONV => RELU => BN operation
            x = Conv2D(f, (3, 3), strides=2, padding="same")(x)
            x = LeakyReLU(alpha=0.2)(x)
            x = BatchNormalization(axis=chanDim)(x)

        # flatten the network and then construct our latent vector
        volumeSize = K.int_shape(x)
        print("volumeSize:",volumeSize)
        x = Flatten()(x)
        print("x shape", K.int_shape(x))
        latent = Dense(latentDim)(x)

        # build the encoder model
        encoder = Model(inputs, latent, name="encoder")

        # start building the decoder model which will accept the
        # output of the encoder as its inputs
        latentInputs = Input(shape=(latentDim,))
        x = Dense(np.prod(volumeSize[1:]))(latentInputs)
        print("prod shape:",np.prod(volumeSize[1:]))
        print("x shape",K.int_shape(x))
        x = Reshape((volumeSize[1], volumeSize[2], volumeSize[3]))(x)
        print("x shape",K.int_shape(x))
        # loop over our number of filters again, but this time in
        # reverse order
        for f in filters[::-1]:
            # apply a CONV_TRANSPOSE => RELU => BN operation
            x = Conv2DTranspose(f, (3, 3), strides=(2, 2),
                padding="same")(x)
            x = LeakyReLU(alpha=0.2)(x)
            x = BatchNormalization(axis=chanDim)(x)

        # apply a single CONV_TRANSPOSE layer used to recover the
        # original depth of the image
        x = Conv2DTranspose(depth, (3, 3), padding="same")(x)
        outputs = Activation("sigmoid")(x)

        # build the decoder model
        decoder = Model(latentInputs, outputs, name="decoder")

        # our autoencoder is the encoder + decoder
        autoencoder = Model(inputs, decoder(encoder(inputs)),
            name="autoencoder")

        # return a 3-tuple of the encoder, decoder, and autoencoder
        return (encoder, decoder, autoencoder)

In [0]:
# set the matplotlib backend so figures can be saved in the background
import matplotlib
matplotlib.use("Agg")

# import the necessary packages
#from pyimagesearch.convautoencoder import ConvAutoencoder
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.datasets import fashion_mnist
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import numpy as np
import random
import pickle
import cv2

In [0]:
def build_unsupervised_dataset(data, labels, validLabel=1,
    anomalyLabel=3, contam=0.01, seed=42):
    # grab all indexes of the supplied class label that are *truly*
    # that particular label, then grab the indexes of the image
    # labels that will serve as our "anomalies"
    validIdxs = np.where(labels == validLabel)[0]
    anomalyIdxs = np.where(labels == anomalyLabel)[0]

    # randomly shuffle both sets of indexes
    random.shuffle(validIdxs)
    random.shuffle(anomalyIdxs)

    # compute the total number of anomaly data points to select
    i = int(len(validIdxs) * contam)
    anomalyIdxs = anomalyIdxs[:i]

    # use NumPy array indexing to extract both the valid images and
    # "anomlay" images
    validImages = data[validIdxs]
    anomalyImages = data[anomalyIdxs]

    # stack the valid images and anomaly images together to form a
    # single data matrix and then shuffle the rows
    images = np.vstack([validImages, anomalyImages])
    np.random.seed(seed)
    np.random.shuffle(images)

    # return the set of images
    return images

In [0]:
def visualize_predictions(decoded, gt, samples=10):
    # initialize our list of output images
    outputs = None

    # loop over our number of output samples
    for i in range(0, samples):
        # grab the original image and reconstructed image
        original = (gt[i] * 255).astype("uint8")
        recon = (decoded[i] * 255).astype("uint8")

        # stack the original and reconstructed image side-by-side
        output = np.hstack([original, recon])

        # if the outputs array is empty, initialize it as the current
        # side-by-side image display
        if outputs is None:
            outputs = output

        # otherwise, vertically stack the outputs
        else:
            outputs = np.vstack([outputs, output])

    # return the output images
    return outputs

In [28]:
# initialize the number of epochs to train for, initial learning rate,
# and batch size
EPOCHS = 20
INIT_LR = 1e-3
BS = 32

# load the MNIST dataset
print("[INFO] loading Fashion MNIST dataset...")
((trainX, trainY), (testX, testY)) = fashion_mnist.load_data()

# build our unsupervised dataset of images with a small amount of
# contamination (i.e., anomalies) added into it
print("[INFO] creating unsupervised dataset...")
images = build_unsupervised_dataset(trainX, trainY, validLabel=2,
    anomalyLabel=5, contam=0.01)

# add a channel dimension to every image in the dataset, then scale
# the pixel intensities to the range [0, 1]
images = np.expand_dims(images, axis=-1)
images = images.astype("float32") / 255.0

# construct the training and testing split
(trainX, testX) = train_test_split(images, test_size=0.2,
    random_state=42)

# construct our convolutional autoencoder
print("[INFO] building autoencoder...")
(encoder, decoder, autoencoder) = ConvAutoencoder.build(28, 28, 1, filters=(64, 128), latentDim=32)
opt = Adam(lr=INIT_LR, decay=INIT_LR / EPOCHS)
autoencoder.compile(loss="mse", optimizer=opt)



[INFO] loading Fashion MNIST dataset...
[INFO] creating unsupervised dataset...
[INFO] building autoencoder...
volumeSize: (None, 7, 7, 128)
x shape (None, 6272)
prod shape: 6272
x shape (None, 6272)
x shape (None, 7, 7, 128)


In [29]:
# train the convolutional autoencoder
H = autoencoder.fit(
    trainX, trainX,
    validation_data=(testX, testX),
    epochs=EPOCHS,
    batch_size=BS)

Epoch 1/20
152/152 [==============================] - 31s 202ms/step - loss: 0.0263 - val_loss: 0.1088
Epoch 2/20
152/152 [==============================] - 31s 202ms/step - loss: 0.0119 - val_loss: 0.0803
Epoch 3/20
152/152 [==============================] - 31s 203ms/step - loss: 0.0100 - val_loss: 0.0474
Epoch 4/20
152/152 [==============================] - 31s 202ms/step - loss: 0.0087 - val_loss: 0.0138
Epoch 5/20
152/152 [==============================] - 31s 201ms/step - loss: 0.0081 - val_loss: 0.0080
Epoch 6/20
152/152 [==============================] - 31s 203ms/step - loss: 0.0077 - val_loss: 0.0080
Epoch 7/20
152/152 [==============================] - 31s 202ms/step - loss: 0.0075 - val_loss: 0.0071
Epoch 8/20
152/152 [==============================] - 31s 202ms/step - loss: 0.0070 - val_loss: 0.0070
Epoch 9/20
152/152 [==============================] - 31s 201ms/step - loss: 0.0068 - val_loss: 0.0070
Epoch 10/20
152/152 [==============================] - 31s 202ms/step - l

In [31]:
# use the convolutional autoencoder to make predictions on the
# testing images, construct the visualization, and then save it
# to disk
print("[INFO] making predictions...")
decoded = autoencoder.predict(testX)
vis = visualize_predictions(decoded, testX)
cv2.imwrite("/content/drive/My Drive/Colab Notebooks/autoencoder/encoder_vis.png", vis)

[INFO] making predictions...


False

In [32]:
img = cv2.imread('/content/drive/My Drive/Colab Notebooks/autoencoder/encoder_vis.png', cv2.IMREAD_UNCHANGED)
cv2_imshow(img)

AttributeError: ignored

In [33]:
# construct a plot that plots and saves the training history
N = np.arange(0, EPOCHS)
plt.style.use("ggplot")
plt.figure()
plt.plot(N, H.history["loss"], label="train_loss")
plt.plot(N, H.history["val_loss"], label="val_loss")
plt.title("Training Loss")
plt.xlabel("Epoch #")
plt.ylabel("Loss")
plt.legend(loc="lower left")
plt.savefig("/content/drive/My Drive/Colab Notebooks/autoencoder/plot.png")

FileNotFoundError: ignored

In [34]:
img = cv2.imread('/content/drive/My Drive/Colab Notebooks/autoencoder/plot.png', cv2.IMREAD_UNCHANGED)
cv2_imshow(img)

AttributeError: ignored

In [35]:
# serialize the image data to disk
print("[INFO] saving image data...")
f = open("/content/drive/My Drive/Colab Notebooks/autoencoder/output/images.pickle", "wb")
f.write(pickle.dumps(images))
f.close()

# serialize the autoencoder model to disk
print("[INFO] saving autoencoder...")
autoencoder.save("/content/drive/My Drive/Colab Notebooks/autoencoder/output/autoencoder.model", save_format="h5")

[INFO] saving image data...


FileNotFoundError: ignored

##**FIND ANOMALIES**

In [36]:
# import the necessary packages
from tensorflow.keras.models import load_model
import numpy as np
#import argparse
import pickle
import cv2

# load the model and image data from disk
print("[INFO] loading autoencoder and image data...")
autoencoder = load_model("/content/drive/My Drive/Colab Notebooks/autoencoder/output/autoencoder.model")
images = pickle.loads(open("/content/drive/My Drive/Colab Notebooks/autoencoder/output/images.pickle", "rb").read())

[INFO] loading autoencoder and image data...


OSError: ignored

In [0]:
# make predictions on our image data and initialize our list of
# reconstruction errors
decoded = autoencoder.predict(images)
errors = []

# loop over all original images and their corresponding
# reconstructions
for (image, recon) in zip(images, decoded):
    # compute the mean squared error between the ground-truth image
    # and the reconstructed image, then add it to our list of errors
    mse = np.mean((image - recon) ** 2)
    errors.append(mse)

In [0]:
# compute the q-th quantile of the errors which serves as our
# threshold to identify anomalies -- any data point that our model
# reconstructed with > threshold error will be marked as an outlier
thresh = np.quantile(errors, 0.999)
idxs = np.where(np.array(errors) >= thresh)[0]
print("[INFO] mse threshold: {}".format(thresh))
print("[INFO] {} outliers found".format(len(idxs)))

# initialize the outputs array
outputs = None

In [0]:
# loop over the indexes of images with a high mean squared error term
for i in idxs:
    # grab the original image and reconstructed image
    original = (images[i] * 255).astype("uint8")
    recon = (decoded[i] * 255).astype("uint8")

    # stack the original and reconstructed image side-by-side
    output = np.hstack([original, recon])

    # if the outputs array is empty, initialize it as the current
    # side-by-side image display
    if outputs is None:
        outputs = output

    # otherwise, vertically stack the outputs
    else:
        outputs = np.vstack([outputs, output])



In [0]:
# show the output visualization
cv2_imshow(outputs)